In [1]:
%load_ext autotime
# %unload_ext autotime

In [2]:
import sys
sys.path.append("..")

In [3]:
import numpy as np
import parselmouth
import pyaudio
from sonorus.audio import VADAudioInputStreamer
from sonorus.audio.utils import audio_float2int
from sonorus.audio.praat import (reduce_noise, 
                                 change_gender, 
                                 change_pitch)
from std_msgs.msg import UInt8MultiArray
from IPython.display import Audio
import librosa
import soundfile as sf
import ipywidgets
import pickle
import wave
from pathlib import Path

In [13]:
male_file = "/home/harold/Downloads/chapter-33.wav"
# male_file = "/home/harold/Downloads/long.wav"
male_file = "/home/harold/Downloads/coffee-01.wav"

In [14]:
audio, sr = librosa.load(male_file, sr=24000, dtype=np.float64)
male_sound = parselmouth.Sound(values=audio, sampling_frequency=sr)
print(male_sound.values, male_sound.sampling_frequency)
Audio(data=male_sound.values, 
      rate=male_sound.sampling_frequency)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  5.83535885e-04
  -6.86211833e-05  0.00000000e+00]] 24000.0


In [15]:
noise_reduced_sound = male_sound
# noise_reduced_sound = reduce_noise(male_sound)
# print(noise_reduced_sound.values, 
#       noise_reduced_sound.sampling_frequency)
# Audio(data=noise_reduced_sound.values, 
#       rate=noise_reduced_sound.sampling_frequency)

In [19]:
female_sound = change_gender(noise_reduced_sound, 
                             formant_shift_ratio=1.12,
                             pitch_range_factor=1.0)
print(female_sound.values, 
      female_sound.sampling_frequency)
Audio(data=female_sound.values, 
      rate=female_sound.sampling_frequency)

# def interactive_change_gender(sound_file=male_file, factor=1.2):
#     sound = parselmouth.Sound(sound_file)
#     sound_changed_gender = change_gender(sound, formant_shift_ratio=factor)
#     return Audio(data=sound_changed_gender.values, rate=sound_changed_gender.sampling_frequency)

# w = ipywidgets.interact(interactive_change_gender,
#                         factor=ipywidgets.FloatSlider(min=0.25, 
#                                                       max=4, 
#                                                       step=0.05, 
#                                                       value=1.2))

[[ 1.02362477e-05 -7.64668141e-06  5.46230009e-06 ... -2.51968101e-06
  -1.80439950e-06  4.18660557e-06]] 24000.0


In [17]:
pitch_sound = change_pitch(noise_reduced_sound, 
                           factor=1.5)
print(pitch_sound.values, 
      pitch_sound.sampling_frequency)
Audio(data=pitch_sound.values, 
      rate=pitch_sound.sampling_frequency)

# def interactive_change_pitch(sound_file=male_file, factor=1.5):
#     sound = parselmouth.Sound(sound_file)
#     sound_changed_pitch = change_pitch(sound, factor)
#     return Audio(data=sound_changed_pitch.values, rate=sound_changed_pitch.sampling_frequency)

# w = ipywidgets.interact(interactive_change_pitch,
#                         factor=ipywidgets.FloatSlider(min=0.25, 
#                                                       max=4, 
#                                                       step=0.05, 
#                                                       value=1.5))

[[ 6.55591534e-05  6.55591534e-05  6.55591534e-05 ...  6.49095039e-04
  -3.06202986e-06  6.55591534e-05]] 24000.0


In [9]:
# sf.write(
#     file=male_file[:-4]+"-pitch-changed.wav",
#     data=audio_float2int(pitch_sound.values)[0],
#     samplerate=sr,
# )

In [10]:
from kaldi.feat.mfcc import Mfcc, MfccOptions
from kaldi.feat.mel import MelBanksOptions
from kaldi.matrix import Vector
from kaldi.gmm import DiagGmm
from kaldi.base.io import istream, ifstream

In [11]:
mel_opts = MelBanksOptions(num_bins=40)
mel_opts.low_freq = 20
mel_opts.high_freq = -400

mfcc_opts = MfccOptions()
mfcc_opts.num_ceps = 40
mfcc_opts.mel_opts = mel_opts
mfcc_opts.use_energy = False

mfcc = Mfcc(mfcc_opts)

In [12]:
mfcc.compute(wave=Vector(pitch_sound.values[0]), vtln_warp=1)


 3.2051e+01 -4.3159e+01 -8.1752e+00  ...  -2.8173e+00 -4.2473e-01  3.1072e+00
 3.3210e+01 -4.4194e+01 -1.5798e+01  ...  -2.2159e+00  7.3852e-01 -1.3971e-01
 3.2652e+01 -4.4653e+01 -8.7988e+00  ...   1.9063e+00  2.2161e-01 -9.1362e-01
                ...                   ⋱                   ...                
 3.3054e+01 -4.5621e+01 -1.0369e+01  ...  -3.9961e-01 -5.2204e+00 -2.2753e+00
 3.3874e+01 -4.2159e+01 -1.0995e+01  ...   7.3188e-01 -7.4643e+00 -3.0499e+00
 3.2454e+01 -4.5156e+01 -1.0835e+01  ...  -4.3573e+00 -1.5550e+00  9.9739e-01
[kaldi.matrix._matrix.Matrix of size 181x40]

In [ ]:
# gmm_path = "/home/harold/Documents/IISc-work/"\
#             "local-speech-generation/kaldi/s5/"\
#             "exp/nnet3_cleaned/extractor/final.dubm"
# stream = ifstream.from_file(gmm_path)
# dubm = DiagGmm.read(stream)

In [ ]:
from sonorus.speech.kaldi.create_confs import (create_mfcc_conf,
                                               create_ivector_extractor_conf)

from sonorus.speech.kaldi import PhonemeSegmenter

In [ ]:
# # root_dir = ""
# root_dir = "/home/harold/Pictures/"

# model_rxfilename = f"{root_dir}exp/tdnn_7b_chain_online/final.mdl"
# graph_rxfilename = f"{root_dir}exp/tdnn_7b_chain_online/graph_pp/HCLG.fst"
# symbols_filename = f"{root_dir}data/lang/words.txt"
# tree_rxfilename = f"{root_dir}exp/tdnn_7b_chain_online/tree"

# lexicon_rxfilename = f"{root_dir}data/lang/L.fst"
# disambig_rxfilename = f"{root_dir}data/lang/phones/disambig.int"

# phones_file = f"{root_dir}data/lang/phones.txt"
# word_boundary = f"{root_dir}data/lang/phones/word_boundary.int"

# mfcc_conf = f"{root_dir}conf/mfcc_hires.conf"
# ivec_conf = f"{root_dir}conf/ivector_extractor.conf"

In [ ]:
root_dir = "/home/harold/Documents/IISc-work/sonorus/examples/librispeech_tgsmall/"
root_dir = Path(root_dir)

model_rxfilename = str(root_dir/"exp/chain_cleaned/tdnn_1d_sp/final.mdl")
graph_rxfilename = str(root_dir/"exp/chain_cleaned/tdnn_1d_sp/graph_tgsmall/HCLG.fst")
symbols_filename = str(root_dir/"exp/chain_cleaned/tdnn_1d_sp/graph_tgsmall/words.txt")
tree_rxfilename = str(root_dir/"exp/chain_cleaned/tdnn_1d_sp/tree")

lexicon_rxfilename = str(root_dir/"data/lang_tgsmall/L.fst")
disambig_rxfilename = str(root_dir/"data/lang_tgsmall/phones/disambig.int")

phoneme_file = str(root_dir/"data/lang_tgsmall/phones.txt")

mfcc_conf = create_mfcc_conf(root_dir) # with default params
ivec_conf = create_ivector_extractor_conf(root_dir) # with default params

In [ ]:
wav_file = "/home/harold/Pictures/data/test/chapter-33-pitch-changed.wav"
wav_scp = "/home/harold/Pictures/data/test/wav.scp"
spk2utt = "/home/harold/Pictures/data/test/spk2utt"

In [ ]:
phoneme_segmenter = PhonemeSegmenter(
    model_rxfilename,
    graph_rxfilename,
    symbols_filename,
    tree_rxfilename,
    lexicon_rxfilename,
    disambig_rxfilename,
    phoneme_file,
    mfcc_conf,
    ivec_conf,
)

In [ ]:
seg = phoneme_segmenter.segment(pitch_sound.values, sample_rate=sr)
seg

In [ ]:
from hr.lip_control import PhonemesPublisher

phoneme_pub = PhonemesPublisher()
phoneme_pub.to_visemes(seg["utt1"]["phonemes"])

In [ ]:
from sonorus.audio import VADAudioInputStreamer

In [ ]:
audio_streamer = VADAudioInputStreamer(
    pa_format=pyaudio.paInt16, 
)

with audio_streamer as streamer:
        for stream in streamer.stream():
            if stream is not None:
                audio = np.frombuffer(stream, dtype=np.int16)
                print(audio.shape, audio[:5])
                phonemes = phoneme_segmenter.segment(
                    audio, sample_rate=streamer.processing_rate)
                print(phonemes)